In [1]:
!pip install diffusers transformers accelerate datasets huggingface_hub

In [ ]:
from huggingface_hub import login
from datasets import load_dataset

dataset = load_dataset("kevinbenoy/anime_random_images", split = "train")


len(dataset)

README.md:   0%|          | 0.00/286 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/523M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/562M [00:00<?, ?B/s]

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())



## Prepare Dataset for Training

### Subtask:
The loaded dataset will be preprocessed for training the diffusion model. This involves resizing images to a standard dimension, normalizing pixel values, and converting them into PyTorch tensors.


In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

def transform(examples):
    examples["pixel_values"] = [preprocess(img.convert("RGB")) for img in examples["image"]]
    return examples

dataset = dataset.with_transform(transform)


In [ ]:
from diffusers import UNet2DModel, DDPMScheduler

# Initialize the UNet2DModel
model = UNet2DModel(
    sample_size=64,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(64, 128, 128, 256),
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


print("UNet2DModel initialized successfully.")

# Initialize the DDPMScheduler
scheduler = DDPMScheduler(num_train_timesteps=400)
print("DDPMScheduler initialized successfully.")


Using device: cpu
UNet2DModel initialized successfully.
DDPMScheduler initialized successfully.


## Configure and Train Diffusion Model

### Subtask:
Configure training parameters and execute the training loop for the diffusion model. This will involve setting up the optimizer, learning rate, and a limited number of training steps. Acknowledge the small dataset size and potential lack of GPU.



In [ ]:
import torch

def custom_collate_fn(batch):
    # Extract 'pixel_values' from each item in the batch
    pixel_values = [item["pixel_values"] for item in batch]
    # Stack them into a single tensor
    return torch.stack(pixel_values)

print("custom_collate_fn defined successfully.")




custom_collate_fn defined successfully.


In [ ]:


# import torch
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from tqdm import tqdm
# from torch.cuda.amp import autocast, GradScaler

# scaler = GradScaler()

# # 2. Define training parameters
# num_epochs = 50
# batch_size = 32
# learning_rate = 1e-4

# # 3. Create a DataLoader from the preprocessed dataset, using the custom collate function
# # train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
# train_dataloader = DataLoader(
#     dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=4,      # or 8 if Colab Pro
#     pin_memory=True,
#     persistent_workers=True,
#     collate_fn=custom_collate_fn)


# # 4. Initialize the AdamW optimizer for the model
# optimizer = AdamW(model.parameters(), lr=learning_rate)

# # 5. Move the model to the 'cpu' device
# model.to(device)

# # 6. Implement the training loop
# model.train() # Set model to training mode

# print(f"Starting training for {num_epochs} epochs on {device}...")

# for epoch in range(num_epochs):
#     epoch_loss = 0
#     pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", leave=False)

#     for batch in pbar:
#         clean_images = batch.to(device)

#         noise = torch.randn(clean_images.shape, device=device)
#         timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (clean_images.shape[0],), device=device)

#         noisy_images = scheduler.add_noise(clean_images, noise, timesteps)

#         with torch.cuda.amp.autocast():
#             noise_pred = model(noisy_images, timesteps).sample
#             loss = torch.nn.functional.mse_loss(noise_pred, noise)

#         optimizer.zero_grad()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#     # for batch in pbar:
#     #     clean_images = batch.to(device)
#     #     noise = torch.randn(clean_images.shape).to(device)

#     #     timesteps = torch.randint(
#     #         0, scheduler.config.num_train_timesteps, (clean_images.shape[0],)
#     #     ).to(device)

#     #     noisy_images = scheduler.add_noise(clean_images, noise, timesteps)
#     #     noise_pred = model(noisy_images, timesteps).sample
#     #     loss = torch.nn.functional.mse_loss(noise_pred, noise)

#     #     optimizer.zero_grad()
#     #     loss.backward()
#     #     optimizer.step()

#         epoch_loss += loss.item()

#         # Show current batch loss
#         pbar.set_postfix({"batch_loss": loss.item()})

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_dataloader):.4f}")

# print("Training complete.")


In [ ]:
from torch.utils.data import DataLoader
from torch.amp import autocast, GradScaler # Updated import
from tqdm import tqdm
import torch
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model.to(device) # Moved model to the correct device
model.train()

num_epochs = 50
batch_size = 32
learning_rate = 1e-4

train_dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    collate_fn=custom_collate_fn
)

optimizer = AdamW(model.parameters(), lr=learning_rate)
scaler = GradScaler('cuda') # Updated to use torch.amp.GradScaler with device


print(f"Starting training for {num_epochs} epochs...")

for epoch in range(num_epochs):
    epoch_loss = 0
    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")

    for batch in pbar:
        clean_images = batch.to(device)

        noise = torch.randn_like(clean_images)
        timesteps = torch.randint(
            0, scheduler.config.num_train_timesteps,
            (clean_images.size(0),), device=device
        )

        noisy_images = scheduler.add_noise(clean_images, noise, timesteps)

        with autocast(device_type='cuda'): # Updated to use torch.amp.autocast with device_type
            noise_pred = model(noisy_images, timesteps).sample
            loss = torch.nn.functional.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

    print(f"Epoch {epoch+1}/{num_epochs}, Avg Loss: {epoch_loss/len(train_dataloader):.4f}")

print("Training complete.")

/tmp/ipython-input-1862396663.py:28: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  scaler = GradScaler('cuda') # Updated to use torch.amp.GradScaler with device


Using device: cpu
Starting training for 50 epochs...


Epoch 1:   0%|          | 0/625 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1: 100%|██████████| 625/625 [2:11:42<00:00, 12.64s/it, loss=0.0763]


Epoch 1/50, Avg Loss: 0.1146


Epoch 2:  14%|█▍        | 86/625 [18:17<1:52:29, 12.52s/it, loss=0.0949]


## Save Trained Model to Hugging Face

### Subtask:
Authenticate your Hugging Face account and save the trained diffusion model to a new repository on your Hugging Face profile.


**Reasoning**:
I will import `create_repo` from `huggingface_hub`, call `login()` to authenticate, define a `repo_id`, create the repository using `create_repo`, and then save the `model` and `scheduler` to that repository.



In [ ]:

from huggingface_hub import login, create_repo

# # Log in to Hugging Face (you will be prompted to enter your token)
# login()

# Define your repository ID
# Replace 'your-username' with your actual Hugging Face username
repo_id = "Tomisin05/unconditional-anime-diffusion-model"

# Create the repository on Hugging Face
create_repo(repo_id, exist_ok=True, repo_type="model")
print(f"Hugging Face repository '{repo_id}' created or already exists.")

# Save the trained model and scheduler to the repository
model.push_to_hub(repo_id)
scheduler.push_to_hub(repo_id)

print("Model and scheduler saved to Hugging Face repository.")


In [ ]:
from diffusers import DDPMPipeline
import torch

pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

# Move model to GPU if supported
pipeline.to("cuda")  # Or change to "cpu" if your model is CPU-trained

num_images_to_generate = 10
batch_size = 8

test_images = []

print(f"Starting generation of {num_images_to_generate} images...")

for i in range(0, num_images_to_generate, batch_size):
    with torch.no_grad():
        images_batch_output = pipeline(
            batch_size=batch_size,
            output_type="pil",
            num_inference_steps=400
        )
    test_images.extend(images_batch_output.images) # Access the .images attribute

    print(f"Generated {len(test_images)} / {num_images_to_generate} images...")

print(f"Successfully generated {len(test_images)} images.")

In [ ]:
import matplotlib.pyplot as plt

# Display a few of the generated images
print(f"Displaying {max(5, len(test_images))} sample generated images:")

plt.figure(figsize=(10, 2))
for i, image in enumerate(test_images[:16]):
    plt.subplot(1, 16, i + 1)
    plt.imshow(image)
    plt.axis('off')
plt.show()

In [ ]:
from diffusers import DDPMPipeline

pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

pipeline.push_to_hub(
    "Tomisin05/unconditional-anime-diffusion-model",
    commit_message="Upload trained DDPM pipeline."
)


## Generate 20,000 Images

### Subtask:
Using the trained diffusion model, generate 20,000 new images. The generation process will be configured to ensure diversity and quality as much as possible, given the model's training limitations.


**Reasoning**:
I will import necessary libraries, instantiate the DDPM pipeline from the previously trained model and scheduler, then iterate to generate 20,000 images in batches, collecting them in a list.



In [ ]:
import os
import torch
from diffusers import DDPMPipeline
from datasets import Dataset, Image

# Create pipeline
pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

# Total images
num_images_to_generate = 21000
batch_size = 8
chunk_size = 100  # how often to push to HuggingFace

save_dir = "generated_images"
os.makedirs(save_dir, exist_ok=True)

# Hugging Face repo
hf_repo = "Tomisin05/generated-anime-images"

print(f"Starting generation of {num_images_to_generate} images...")

global_count = 0  # total images generated so far
current_chunk_files = []  # list of files waiting to be uploaded

for i in range(0, num_images_to_generate, batch_size):

    # generate batch of images
    with torch.no_grad():
        images = pipeline(
            batch_size=batch_size,
            output_type="pil",
            num_inference_steps=400
        ).images

    # save images and track filenames
    for img in images:
        filename = f"{save_dir}/{global_count}.png"
        img.save(filename)
        current_chunk_files.append(filename)
        global_count += 1

    # If 100 images collected → push to hub
    if len(current_chunk_files) >= chunk_size:
        print(f"Pushing {len(current_chunk_files)} images to Hugging Face...")

        # Build HF Dataset just for this chunk
        ds = Dataset.from_dict({"image": current_chunk_files}).cast_column("image", Image())

        ds.push_to_hub(
            hf_repo,
            commit_message=f"Upload {global_count} images",
            private=True
        )

        print(f"Uploaded {len(current_chunk_files)} images")

        # Clear list
        current_chunk_files = []

# After loop: upload any remaining images
if current_chunk_files:
    print(f"Pushing final {len(current_chunk_files)} images...")
    ds = Dataset.from_dict({"image": current_chunk_files}).cast_column("image", Image())
    ds.push_to_hub(
        hf_repo,
        commit_message=f"Final upload - total {global_count}",
        private=True
    )

print(f"Done! Total uploaded: {global_count} images.")


In [ ]:


# import os

# # Create a DDPMPipeline object from the trained model and scheduler
# pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

# # Define the total number of images to generate and a suitable batch_size
# num_images_to_generate = 21000
# batch_size = 8  # Adjust based on memory availability

# os.makedirs("generated_images", exist_ok=True)


# print(f"Starting generation of {num_images_to_generate} images...")

# count = 0
# for i in range(0, num_images_to_generate, batch_size):
#     with torch.no_grad():
#         images = pipeline( batch_size=batch_size,
#             output_type="pil",
#             num_inference_steps=400  ).images
#     for img in images:
#         img.save(f"generated_images/{count}.png")
#         count += 1


# # Print a confirmation message
# print(f"Successfully generated {count} images.")





## Display Sample Generated Images

### Subtask:
Display a few of the recently generated images from the `generated_images` list to visualize the current output of the diffusion model.



In [ ]:

import matplotlib.pyplot as plt

import glob
from PIL import Image

sample_files = sorted(glob.glob("generated_images/*.png"))[:5]
images = [Image.open(f) for f in sample_files]

# Display a few of the generated images
print(f"Displaying {min(5, len(images))} sample generated images:")

plt.figure(figsize=(10, 2))
for i, image in enumerate(images):
    plt.subplot(1, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
plt.show()



## Create and Push Generated Image Dataset to Hugging Face

### Subtask:
The previously generated images will be compiled into a new Hugging Face dataset. This dataset will then be uploaded to a new repository on your Hugging Face profile, making it accessible for future use.


**Reasoning**:
I will import the `Dataset` class, create a dictionary from the `generated_images`, instantiate a Hugging Face Dataset, define a new repository ID, and then push the dataset to the Hugging Face Hub.



In [ ]:
import datasets
from datasets import Dataset, Image


ds = Dataset.from_dict(
    {"image": [f"generated_images/{i}.png" for i in range(global_count)]}
).cast_column("image", Image())

ds.push_to_hub("Tomisin05/generated-anime-images", commit_message=f"Upload {global_count}  images", private=True)


